In [255]:
import pandas as pd
import numpy as np


import panel as pn
pn.extension('vega')

import matplotlib.pyplot as plt

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Range1d
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_8
from bokeh.models import Legend, LegendItem
from bokeh.palettes import Category20
from bokeh.embed import file_html
from bokeh.resources import CDN
import copy
import requests
import json
from datetime import timedelta, date, datetime

# Read in and prepare the latest COVID data

In [256]:
data = requests.get('http://pomber.github.io/covid19/timeseries.json').json()

In [257]:
#Generate dataframe from json data
keys_countries= data.keys() 
df=pd.DataFrame()
for key in data:
    df1= pd.json_normalize(data[key])
    df1['Country']=key
    df= df.append(df1)

In [258]:
# daily cases per country (above is cumulatve)
keys_countries= data.keys() 
df_daily=pd.DataFrame()
for key in data:
    df1= pd.json_normalize(data[key])
    df1.set_index(['date'],inplace=True)
    df2=copy.deepcopy(df1)
    for col in df2.columns:
        df2[col]=df2[col]- df2[col].shift(1)
    df2['Country']=key
    df_daily= df_daily.append(df2)
df_daily.reset_index(inplace=True)

In [259]:

df_daily.date = pd.to_datetime(df_daily.date)
df.date = pd.to_datetime(df.date)


In [135]:
## Add data adjusted for population. 
url = 'https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_Population/WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx'
df_pops = pd.read_excel(url, sheet_name='ESTIMATES',engine='openpyxl', header=16)[['Region, subregion, country or area *','2020']]

In [136]:
df_pops.columns=['Country','Population']
df_pops['Population']= df_pops['Population']*1000

In [250]:
cumulative_df_pops = pd.merge(df, df_pops, on='Country')
daily_df_pops = pd.merge(df_daily, df_pops, on='Country')


In [251]:
#consistent column order for daily and cumulative dfs
col_order=cumulative_df_pops.columns
daily_df_pops = daily_df_pops[col_order]


In [139]:
cumulative_df_pops.date=pd.to_datetime(cumulative_df_pops.date) 


In [246]:
per100Th_cumulative = copy.deepcopy(cumulative_df_pops)

for col in col_order[1:4]:
    per100Th_cumulative[col]= per100Th_cumulative[col]*100000/(per100Th_cumulative['Population']).apply(pd.to_numeric, errors='coerce')

In [245]:
per100Th_daily=copy.deepcopy(daily_df_pops)
for col in col_order[1:4]:
    per100Th_daily[col]= per100Th_daily[col]*100000/(per100Th_daily['Population']).apply(pd.to_numeric, errors='coerce')


## Create widget+plot pane

In [274]:
countries = sorted(pd.unique(per100Th_daily['Country']))
country = pn.widgets.Select(name='Country', options=countries)
parameters = ['confirmed','deaths', 'recovered']
#parameters.append('all')
parameter = pn.widgets.Select(name='parameter', options=parameters)
PHT = pn.widgets.RadioButtonGroup(name='Per hundred thousand', options= ['Per hundred thousand','Total Number'], button_type ='success')
daily = pn.widgets.RadioButtonGroup(name='daily', options= ['day change','cumulative'], button_type ='success')
aggregated =pn.widgets.RadioButtonGroup(name='aggregated', options= ['daily value','7-day average'], button_type ='success')

title = '# Covid data over time'
subtitle = 'Cases or death by country'

@pn.depends(country.param.value, parameter.param.value, PHT.param.value, daily.param.value, aggregated.param.value)
def get_plot(country, parameter, PHT, daily, aggregated): 
    if daily== 'day change' and PHT == 'Per hundred thousand':
        chosen = copy.deepcopy(per100Th_daily.dropna())
    if daily == 'cumulative' and PHT == 'Per hundred thousand':
        chosen = copy.deepcopy(per100Th_cumulative)
    if daily == 'day change' and PHT == 'Total Number':
        chosen = copy.deepcopy(df_daily.dropna())
    if daily== 'cumulative' and PHT=='Total Number':
        chosen = copy.deepcopy(df)
    group2 =chosen[chosen['Country']==country][[parameter, 'date']]
    if aggregated=='7-day average':
        group2=group2.set_index('date').resample('7D').mean()
        group2.reset_index(inplace=True)
  
    else:
            group2=group2
    cds = ColumnDataSource(group2)
    hover = HoverTool(
            tooltips=[
                ("Country", "$Country"),
                ('date','@date')
        ]
    )
    p = figure(plot_width=500, plot_height=300, tools=[hover],x_axis_type='datetime')
    p.line('date', parameter, source=cds)
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = parameter
    #p.x_range = Range1d(1, 8, bounds=(1, 8))
    #p.y_range = Range1d(10, 50, bounds=(10, 50))
    p.title.text = "Country time series"
    p.title.align = "center"
    p.title.text_font_size = "12px"
    return p


newObj=pn.Column(title, 
pn.Row(pn.Column(subtitle, country, parameter, PHT, daily, aggregated), get_plot))

In [275]:

newObj

Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] Select(name='Country', options=['Afghanistan', ...], value='Afghanistan')
            [2] Select(name='parameter', options=['confirmed', ...], value='confirmed')
            [3] RadioButtonGroup(button_type='success', name='Per hundred thousand', options=['Per hundred thousand', ...], value='Per hundred thousand')
            [4] RadioButtonGroup(button_type='success', name='daily', options=['day change', ...], value='day change')
            [5] RadioButtonGroup(button_type='success', name='aggregated', options=['daily value', ...], value='daily value')
        [1] ParamFunction(function)